# Bee and hornet

## email

Here is a link to the genome assemblies that we did: https://www.dropbox.com/sh/8leaqum6nnvrein/AADBQfsW7UvLZI7ZBfsDs1sza?dl=0   The assemblies are a mix of Oxford Nanopore and Illumina and are pretty good.   It would probably be best if your postdoc could run MAKER2 to do the annotation and BUSCO: https://busco.ezlab.org/ which gives a good picture of assembly completeness

## run maker2

### install programs

On Elf server, creat environment with `conda create -p /gpfs/gpfs/scratch/xc278/maker2`

install genemark-es manually. link the file to the bin path of the environment 
`ln -s /gpfs/gpfs/scratch/xc278/p/gm_et_linux_64/gmes_petap/gmhmme3 /gpfs/gpfs/scratch/xc278/maker2/bin/`

fix problem with repeatmasker `https://wiki.hpcc.msu.edu/display/ITH/Installing+maker+using+conda?preview=/29655183/29655184/repeatmasker-small-copy.mp4`

```bash
source activate /gpfs/gpfs/scratch/xc278/maker2
cd /gpfs/gpfs/scratch/xc278/maker2/share/RepeatMasker
perl ./configure
```
finish as in the video

### Trinity assembly for Vespa mandarinia

https://www.ncbi.nlm.nih.gov/sra/?term=SRP064879

SRR2664950

Note: need to install Trinity, bowtie2 and other required programs. `conda bioconda` installed Trinity seems not working

```bash
source activate bio
cd /home/scratch
wget ftp://ftp-trace.ncbi.nih.gov/sra/sra-instant/reads/ByRun/sra/SRR/SRR266/SRR2664950/SRR2664950.sra
fastq-dump --split-files '@$sn[_$rn]/$ri' -O ./ SRR2664950.sra

Trinity  --seqType fq  --max_memory 240G --CPU 36 --trimmomatic --no_normalize_reads --output /home/scratch/Trinity/  --left  SRR2664950_1.fastq   --right SRR2664950_2.fastq


```

### prepare reference sequences

download swissprot sequence from uniprot website

for bee, choose protein sequences from the Apis genus as reference. download protein and transcript models from three available species from the link `https://www.ncbi.nlm.nih.gov/genome/?term=txid7459[Organism:exp]`
* Apis mellifera
* Apis cerana
* Apis dorsata
* Apis florea

for hornet, choose protein sequences from the Vespidae family as references. download protein and transcript sequences of available species from the link `https://www.ncbi.nlm.nih.gov/genome/?term=txid7438[Organism:exp]`
* Polistes dominula
* Polistes canadensis


reference sequences are 
```
GCF_000184785.2_Aflo_1.0_protein.faa
GCF_000184785.2_Aflo_1.0_rna.fna
GCF_000469605.1_Apis_dorsata_1.3_protein.faa
GCF_000469605.1_Apis_dorsata_1.3_rna.fna
GCF_001313835.1_ASM131383v1_protein.faa
GCF_001313835.1_ASM131383v1_rna.fna
GCF_001442555.1_ACSNU-2.0_protein.faa
GCF_001442555.1_ACSNU-2.0_rna.fna
GCF_001465965.1_Pdom_r1.2_protein.faa
GCF_001465965.1_Pdom_r1.2_rna.fna
GCF_003254395.2_Amel_HAv3.1_protein.faa
GCF_003254395.2_Amel_HAv3.1_rna.fna
te_proteins.fasta
uniprot_sprot.fasta
```

combine the protein sequences to file `/gpfs/gpfs/staging/jx76-003/2019Bee/ref/ref.proteins.faa` and rna sequences to `/gpfs/gpfs/staging/jx76-003/2019Bee/ref/ref.rna.faa`

TE sequences (transposobale elements)
`https://github.com/Yandell-Lab/RepeatRunner/blob/master/sample_data/te_proteins.fasta`

#### clean sequences, some nt sequences contains unwanted letters
combine reference protein and rna sequences

```python
import os

folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/'
files_pr = '''GCF_000184785.2_Aflo_1.0_protein.faa
GCF_000469605.1_Apis_dorsata_1.3_protein.faa
GCF_001313835.1_ASM131383v1_protein.faa
GCF_001442555.1_ACSNU-2.0_protein.faa
GCF_001465965.1_Pdom_r1.2_protein.faa
GCF_003254395.2_Amel_HAv3.1_protein.faa
uniprot_sprot.fasta
'''.split()
os.chdir(folder)

file_pr = 'ref.proteins.faa'
from Bio import SeqIO
fout = open(file_pr,'w')
for f in files_pr:
    for s in SeqIO.parse(f,'fasta'):
        if 'X' in str(s.seq):
            continue
        else:
            fout.write('>'+s.id+'\n'+str(s.seq)+'\n')
fout.close()

files_nt = '''GCF_000184785.2_Aflo_1.0_rna.fna
GCF_000469605.1_Apis_dorsata_1.3_rna.fna
GCF_001313835.1_ASM131383v1_rna.fna
GCF_001442555.1_ACSNU-2.0_rna.fna
GCF_001465965.1_Pdom_r1.2_rna.fna
GCF_003254395.2_Amel_HAv3.1_rna.fna
'''.split()
file_nt = 'ref.rna.faa'
fout = open(file_nt,'w')
for f in files_nt:
    for s in SeqIO.parse(f,'fasta'):
        seq = str(s.seq)
        seq = seq.upper()
        if seq.count('A')+seq.count('T')+seq.count('C')+seq.count('G')+seq.count('*')+seq.count('N') == len(seq):
            fout.write('>'+s.id+'\n'+str(s.seq)+'\n')
        else:
            print(s.id,'abnormal',s.seq)
fout.close()
```

### run maker2 for bee

do with the steps listed here `https://reslp.github.io/blog/My-MAKER-Pipeline/`
IO is a limitation, run the work in /fastio/xc278/ then copy the data back

#### step1: first round of maker2

```bash
srun -N 1 --partition long --qos long-award   -t 240:00:00 --pty bash #allocate resource from Elf
source activate /gpfs/gpfs/scratch/xc278/maker2 #activate environment
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/ # go to the working directory
mkdir step1
cd step1
maker -CTL
```

modify the maker_opts.ctl file to 
```
#-----Genome (these are always required)
genome=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/beeScaffolds.fasta #genome sequence (fasta file or fasta embeded in GFF3 file)
organism_type=eukaryotic #eukaryotic or prokaryotic. Default is eukaryotic

#-----Re-annotation Using MAKER Derived GFF3
maker_gff= #MAKER derived GFF3 file
est_pass=0 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=0 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no
protein_pass=0 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=0 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

#-----EST Evidence (for best results provide a file for at least one)
est=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/GCF_000469605.1_Apis_dorsata_1.3_rna.fna #set of ESTs or assembled mRNA-seq in fasta format
altest=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/ref.rna.faa #EST/cDNA sequence file in fasta format from an alternate organism
est_gff= #aligned ESTs or mRNA-seq from an external GFF3 file
altest_gff= #aligned ESTs from a closly relate species in GFF3 format

#-----Protein Homology Evidence (for best results provide a file for at least one)
protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/ref.proteins.faa  #protein sequence file in fasta format (i.e. from mutiple oransisms)
protein_gff=  #aligned protein homology evidence from an external GFF3 file

#-----Repeat Masking (leave values blank to skip repeat masking)
model_org=all #select a model organism for RepBase masking in RepeatMasker
rmlib= #provide an organism specific repeat library in fasta format for RepeatMasker
repeat_protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/te_proteins.fasta #provide a fasta file of transposable element proteins for RepeatRunner
rm_gff= #pre-identified repeat elements from an external GFF3 file
prok_rm=0 #forces MAKER to repeatmask prokaryotes (no reason to change this), 1 = yes, 0 = no
softmask=1 #use soft-masking rather than hard-masking in BLAST (i.e. seg and dust filtering)

#-----Gene Prediction
snaphmm= #SNAP HMM file
gmhmm= #GeneMark HMM file
augustus_species= #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no
trna=1 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no

#-----Other Annotation Feature Types (features MAKER doesn't recognize)
other_gff= #extra features to pass-through to final MAKER generated GFF3 file

#-----External Application Behavior Options
alt_peptide=C #amino acid used to replace non-standard amino acids in BLAST databases
cpus=1 #max number of cpus to use in BLAST and RepeatMasker (not for MPI, leave 1 when using MPI)

#-----MAKER Behavior Options
max_dna_len=100000 #length for dividing up contigs into chunks (increases/decreases memory usage)
min_contig=1 #skip genome contigs below this length (under 10kb are often useless)

pred_flank=200 #flank for extending evidence clusters sent to gene predictors
pred_stats=0 #report AED and QI statistics for all predictions as well as models
AED_threshold=1 #Maximum Annotation Edit Distance allowed (bound by 0 and 1)
min_protein=0 #require at least this many amino acids in predicted proteins
alt_splice=0 #Take extra steps to try and find alternative splicing, 1 = yes, 0 = no
always_complete=0 #extra steps to force start and stop codons, 1 = yes, 0 = no
map_forward=0 #map names and attributes forward from old GFF3 genes, 1 = yes, 0 = no
keep_preds=0 #Concordance threshold to add unsupported gene prediction (bound by 0 and 1)

split_hit=10000 #length for the splitting of hits (expected max intron size for evidence alignments)
single_exon=0 #consider single exon EST evidence when generating annotations, 1 = yes, 0 = no
single_length=250 #min length required for single exon ESTs if 'single_exon is enabled'
correct_est_fusion=0 #limits use of ESTs in annotation to avoid fusion genes

tries=2 #number of times to try a contig if there is a failure for some reason
clean_try=0 #remove all data from previous run before retrying, 1 = yes, 0 = no
clean_up=0 #removes theVoid directory with individual analysis files, 1 = yes, 0 = no
TMP= #specify a directory other than the system default temporary directory for temporary files

```


Note, bee is Apis dorsata  
Hornet is Vespa mandarinia  
changed part:
* genome=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/beeScaffolds.fasta #genome sequence (fasta file or fasta embeded in GFF3 file)
* est=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/GCF_000469605.1_Apis_dorsata_1.3_rna.fna #set of ESTs or assembled mRNA-seq in fasta format
* altest=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/ref.rna.faa #EST/cDNA sequence file in fasta format from an alternate organism
* protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/ref.proteins.faa  #protein sequence file in fasta format (i.e. from mutiple oransisms)
* est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
* protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no

maker_exe.ctl
```
#-----Location of Executables Used by MAKER/EVALUATOR
makeblastdb=/gpfs/gpfs/scratch/xc278/maker2/bin/makeblastdb #location of NCBI+ makeblastdb executable
blastn=/gpfs/gpfs/scratch/xc278/maker2/bin/blastn #location of NCBI+ blastn executable
blastx=/gpfs/gpfs/scratch/xc278/maker2/bin/blastx #location of NCBI+ blastx executable
tblastx=/gpfs/gpfs/scratch/xc278/maker2/bin/tblastx #location of NCBI+ tblastx executable
formatdb= #location of NCBI formatdb executable
blastall= #location of NCBI blastall executable
xdformat= #location of WUBLAST xdformat executable
blasta= #location of WUBLAST blasta executable
RepeatMasker=/gpfs/gpfs/scratch/xc278/maker2/bin/RepeatMasker #location of RepeatMasker executable
exonerate=/gpfs/gpfs/scratch/xc278/maker2/bin/exonerate #location of exonerate executable

#-----Ab-initio Gene Prediction Algorithms
snap=/gpfs/gpfs/scratch/xc278/maker2/bin/snap #location of snap executable
gmhmme3=/gpfs/gpfs/scratch/xc278/maker2/bin/gmhmme3 #location of eukaryotic genemark executable
gmhmmp= #location of prokaryotic genemark executable
augustus=/gpfs/gpfs/scratch/xc278/maker2/bin/augustus #location of augustus executable
fgenesh= #location of fgenesh executable
tRNAscan-SE=/gpfs/gpfs/scratch/xc278/maker2/bin/tRNAscan-SE #location of trnascan executable
snoscan=/gpfs/gpfs/scratch/xc278/maker2/bin/snoscan #location of snoscan executable

#-----Other Algorithms
probuild=/gpfs/gpfs/scratch/xc278/p/gm_et_linux_64/gmes_petap/probuild #location of probuild executable (required for genemark)

```

split the genome to pieces ~100kb and run maker2, generate small scripts to run together
```python

folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1/' #folder of oringal maker
threads = 24 # number of threads

pre_cmd = 'source activate /gpfs/gpfs/scratch/xc278/maker2 ' # commond to run before each individual commands
pyMultiple = '/home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py'

import os
from Bio import SeqIO

#get setting files
file_maker_exe = os.path.join(folder,'maker_exe.ctl')
file_maker_opts = os.path.join(folder, 'maker_opts.ctl')
file_maker_bopts = os.path.join(folder,'maker_bopts.ctl')

#generate workfolder
workfolder = os.path.join(folder,'maker_split_run')
if not os.path.exists(workfolder):
    os.makedirs(workfolder)

#get genome location
for line in open(file_maker_opts):
    if line.startswith('genome='):
        break
file_genome = line[7:].split('#')[0].strip()

#split genome sequences to parts so that each parts contains is longer than 100kb
#remove contigs shorter than 200bp
seqfolder = os.path.join(workfolder,'seq')
if not os.path.exists(seqfolder):
    os.makedirs(seqfolder)

n = 0
l = 0
outfolder = os.path.join(seqfolder,str(n))
if not os.path.exists(outfolder):
    os.makedirs(outfolder)
outfile = open(os.path.join(outfolder,str(n)),'w')
for s in SeqIO.parse(file_genome,'fasta'):
    slen = len(s.seq)
    if slen <=200:
        continue
    l += slen
    outfile.write('>'+s.id+'\n'+str(s.seq)+'\n')
    if l >= 100000:
        outfile.close()
        n += 1
        l = 0
        outfolder = os.path.join(seqfolder,str(n))
        if not os.path.exists(outfolder):
            os.makedirs(outfolder)
        outfile = open(os.path.join(outfolder,str(n)),'w')
outfile.close()

#prepare to run maker for each part
total_seg = n+1
commands = os.path.join(workfolder,'cmds')
fout = open(commands,'w')
for n in range(total_seg):
    for f in [file_maker_exe, file_maker_opts,file_maker_bopts]:
        outfolder = os.path.join(seqfolder,str(n))
        genome = os.path.join(outfolder,str(n))
        os.system(f'cp {f} {outfolder}/')
    fout.write(pre_cmd + '  &&  ' + f'cp -af {outfolder} /home/scratch/ && cd /home/scratch/{n} && maker -genome {genome} -cpus 1  && gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *all.gff {outfolder}/ && rm -rf /home/scratch/{n} \n')
fout.close()
```

run the scripts manually in multiple nodes

```bash
ssh  -o "ServerAliveInterval 60" xc278@elf.rdi2.rutgers.edu
srun -N 16 --partition long --qos long-award   -t 240:00:00 --pty bash #run twice to get 32 working nodes
python /home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py -t 36 -n e3c-262,e3c-263,e3c-264,e3c-265,e3c-266,e3c-267,e3c-268,e3c-269,e3c-270,e3c-271,e3c-272,e3c-273,e3c-274,e3c-275,e3c-276 -s 60 -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1/maker_split_run/cmds &
```

combine the result (discard. use python scripts below)
```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1
for file in $(ls maker_split_run/seq/*/*.gff)
do
    cat $file >> step1.all.gff
done

```

Note, to be compatible with the following step, use python to combine the result

```python
import glob
files = glob.glob('/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1/maker_split_run/seq/*/*.gff')
outfile = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1/step1.all.gff'

ls_gff = []
ls_fasta = []
for f in files:
    txt = open(f).read()
    txt = txt.replace('##gff-version 3\n','',1)
    txt_gff, txt_fa = txt.split('##FASTA\n')
    ls_gff.append(txt_gff)
    ls_fasta.append(txt_fa)

fout = open(outfile,'w')
fout.write('##gff-version 3\n')
fout.write(''.join(ls_gff))
fout.write('##FASTA\n')
fout.write(''.join(ls_fasta))
fout.close()
```

#### step2 training SNAP

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1
maker2zff step1.all.gff
fathom genome.ann genome.dna -validate > snap_validate_output.txt
cat snap_validate_output.txt | grep "error" #MODEL5240 with error, remove it
grep -vwE "MODEL5240" genome.ann > genome.ann2
cat genome.ann2 |grep "MODEL5240"
cat genome.ann |grep "MODEL5240"
fathom genome.ann2 genome.dna -validate
fathom genome.ann genome.dna -categorize 1000
fathom uni.ann uni.dna -export 1000 -plus 
forge export.ann export.dna
hmm-assembler.pl my_genome . > my_genome.hmm
```

#### step3 Run MAKER with the training results from Step 2

Change the following line in the maker_opts.ctl file:

snaphmm=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1/my_genome.hmm #SNAP HMM file

To base the predictions in the second MAKER run only on SNAP remove the filepaths to the protein and est evidence or set the flags for est2genome=0 and protein2genome=0.

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3
maker -CTL
```

change maker_opts.ctl to
```
#-----Genome (these are always required)
genome=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/beeScaffolds.fasta #genome sequence (fasta file or fasta embeded in GFF3 file)
organism_type=eukaryotic #eukaryotic or prokaryotic. Default is eukaryotic

#-----Re-annotation Using MAKER Derived GFF3
maker_gff= #MAKER derived GFF3 file
est_pass=0 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=0 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no
protein_pass=0 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=0 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

#-----EST Evidence (for best results provide a file for at least one)
est=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/GCF_000469605.1_Apis_dorsata_1.3_rna.fna #set of ESTs or assembled mRNA-seq in fasta format
altest=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/ref.rna.faa #EST/cDNA sequence file in fasta format from an alternate organism
est_gff= #aligned ESTs or mRNA-seq from an external GFF3 file
altest_gff= #aligned ESTs from a closly relate species in GFF3 format

#-----Protein Homology Evidence (for best results provide a file for at least one)
protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/ref.proteins.faa  #protein sequence file in fasta format (i.e. from mutiple oransisms)
protein_gff=  #aligned protein homology evidence from an external GFF3 file

#-----Repeat Masking (leave values blank to skip repeat masking)
model_org=all #select a model organism for RepBase masking in RepeatMasker
rmlib= #provide an organism specific repeat library in fasta format for RepeatMasker
repeat_protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/te_proteins.fasta #provide a fasta file of transposable element proteins for RepeatRunner
rm_gff= #pre-identified repeat elements from an external GFF3 file
prok_rm=0 #forces MAKER to repeatmask prokaryotes (no reason to change this), 1 = yes, 0 = no
softmask=1 #use soft-masking rather than hard-masking in BLAST (i.e. seg and dust filtering)

#-----Gene Prediction
snaphmm=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1/my_genome.hmm #SNAP HMM file
gmhmm= #GeneMark HMM file
augustus_species= #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
est2genome=0 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=0 #infer predictions from protein homology, 1 = yes, 0 = no
trna=1 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no

#-----Other Annotation Feature Types (features MAKER doesn't recognize)
other_gff= #extra features to pass-through to final MAKER generated GFF3 file

#-----External Application Behavior Options
alt_peptide=C #amino acid used to replace non-standard amino acids in BLAST databases
cpus=1 #max number of cpus to use in BLAST and RepeatMasker (not for MPI, leave 1 when using MPI)

#-----MAKER Behavior Options
max_dna_len=100000 #length for dividing up contigs into chunks (increases/decreases memory usage)
min_contig=1 #skip genome contigs below this length (under 10kb are often useless)

pred_flank=200 #flank for extending evidence clusters sent to gene predictors
pred_stats=0 #report AED and QI statistics for all predictions as well as models
AED_threshold=1 #Maximum Annotation Edit Distance allowed (bound by 0 and 1)
min_protein=0 #require at least this many amino acids in predicted proteins
alt_splice=0 #Take extra steps to try and find alternative splicing, 1 = yes, 0 = no
always_complete=0 #extra steps to force start and stop codons, 1 = yes, 0 = no
map_forward=0 #map names and attributes forward from old GFF3 genes, 1 = yes, 0 = no
keep_preds=0 #Concordance threshold to add unsupported gene prediction (bound by 0 and 1)

split_hit=10000 #length for the splitting of hits (expected max intron size for evidence alignments)
single_exon=0 #consider single exon EST evidence when generating annotations, 1 = yes, 0 = no
single_length=250 #min length required for single exon ESTs if 'single_exon is enabled'
correct_est_fusion=0 #limits use of ESTs in annotation to avoid fusion genes

tries=2 #number of times to try a contig if there is a failure for some reason
clean_try=0 #remove all data from previous run before retrying, 1 = yes, 0 = no
clean_up=0 #removes theVoid directory with individual analysis files, 1 = yes, 0 = no
TMP= #specify a directory other than the system default temporary directory for temporary files
```


generate scripts
```python
total_seg = 1940
commands = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/cmds.txt'
fout = open(commands,'w')
for n in range(1940):
    cmd = f'source activate /gpfs/gpfs/scratch/xc278/maker2 && cd /home/scratch && mkdir {n} && cd {n} && cp /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/maker*.ctl ./ && maker -genome /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1/maker_split_run/seq/{n}/{n} -cpus 1 &&  gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *.gff /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/maker_split_run/ && rm -rf /home/scratch/{n} \n'
    fout.write(cmd)
fout.close()
```

run scripts
```bash
srun -N 16 --partition long --qos long-award   -t 240:00:00 --pty bash #run twice to get 32 working nodes
python /home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py -t 36 -n e3c-113,e3c-114,e3c-115,e3c-116,e3c-117,e3c-118,e3c-119,e3c-120,e3c-121,e3c-122,e3c-123,e3c-124,e3c-125,e3c-126,e3c-127,e3c-128,e3c-262,e3c-263,e3c-264,e3c-265,e3c-266,e3c-267,e3c-268,e3c-269,e3c-270,e3c-271,e3c-272,e3c-273,e3c-274,e3c-275,e3c-276,e3c-277 -s 60 -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/cmds.txt &
```

the caliburn HPC is not working normally. Some jobs died. Found the dead jobs and run them again with Elf

```python
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/maker_split_run'
cmds = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/cmds_missing.txt'

import os
finished = os.listdir(folder)
#change output.all.gff to 0.all.gff
finished = [0] + [int(e.split('.')[0]) for e in finished]
missing = [e for e in range(1940) if e not in finished]

fout = open(cmds,'w')
for n in missing:
    cmd = f'source activate /gpfs/gpfs/scratch/xc278/maker2 && cd /home/scratch && mkdir bee{n} && cd bee{n} && cp /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/maker*.ctl ./ && maker -genome /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1/maker_split_run/seq/{n}/{n} -cpus 6 &&  gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *.gff /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/maker_split_run/ && rm -rf /home/scratch/bee{n} \n'
    fout.write(cmd)
fout.close()
```

run the scripts
```bash
python /home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py -t 16 -n e1c-051,e1c-052,e1c-053,e1c-054,e1c-055,e1c-056,e1c-057,e1c-058,e1c-059,e1c-060,e1c-061,e1c-062,e1c-063,e1c-064,e1c-065,e1c-066,e1c-109,e1c-110,e1c-111,e1c-112,e1c-113,e1c-114,e1c-115,e1c-116,e1c-117,e1c-118,e1c-119,e1c-120,e1c-121,e1c-122,e1c-123,e1c-124 -s 60 -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/cmds_missing.txt &
```

collect data

```python
import glob
files = glob.glob('/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/maker_split_run/*.gff')
outfile = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/step3.all.gff'

ls_gff = []
ls_fasta = []
for f in files:
    txt = open(f).read()
    txt = txt.replace('##gff-version 3\n','',1)
    txt_gff, txt_fa = txt.split('##FASTA\n')
    ls_gff.append(txt_gff)
    ls_fasta.append(txt_fa)

fout = open(outfile,'w')
fout.write('##gff-version 3\n')
fout.write(''.join(ls_gff))
fout.write('##FASTA\n')
fout.write(''.join(ls_fasta))
fout.close()
```

#### step4 Train SNAP a second time with results from Step 3

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3
maker2zff step3.all.gff
fathom genome.ann genome.dna -validate > snap_validate_output.txt
cat snap_validate_output.txt | grep "error" #MODEL1963 MODEL5311 with error, remove them

grep -vwE "MODEL1963" genome.ann > genome.ann2
grep -vwE "MODEL5311" genome.ann2 >genome.ann3
fathom genome.ann3 genome.dna -validate
fathom genome.ann3 genome.dna -categorize 1000
fathom uni.ann uni.dna -export 1000 -plus 
forge export.ann export.dna
hmm-assembler.pl my_genome . > my_genome.hmm
```

#### step6 train augustus
download the zff2augustus_gbk.pl from `https://github.com/hyphaltip/genome-scripts/blob/master/gene_prediction/zff2augustus_gbk.pl`. install bioperl with `cpanm Bio::Perl`
```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3
perl /gpfs/gpfs/scratch/xc278/maker2/bin/zff2augustus_gbk.pl > augustus.gbk
randomSplit.pl augustus.gbk 100
new_species.pl --species=bee20190616
etraining --species=bee20190616 augustus.gbk.train
augustus --species=bee20190616 augustus.gbk.test | tee first_training.out
optimize_augustus.pl --species=bee20190616 augustus.gbk.train --cpus=34 --kfold=34

```

#### step 8 train geneMark

```
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step8/

#install required module if geneMark does not work
source activate /gpfs/gpfs/scratch/xc278/maker2
cpan install Logger::Simple
cpan install YAML::XS
export PERL5LIB=/gpfs/gpfs/scratch/xc278/maker2/lib/site_perl/5.26.2
export PERL5LIB=/gpfs/gpfs/scratch/xc278/p/braker2/lib/site_perl/5.26.2

/gpfs/gpfs/scratch/xc278/maker2/bin/perl -I /gpfs/gpfs/scratch/xc278/maker2/lib/site_perl/5.26.2 /gpfs/gpfs/scratch/xc278/p/gm_et_linux_64/gmes_petap/gmes_petap.pl --cores 36 --ES --sequence /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/beeScaffolds.fasta
```


tried several hours. cannot get genemark work on caliburn. Tested on ALU and it works.
install the required perl packages and genemark

```bash
cd /lab02/Data_Raw/Xiaolong/20190616CooperationBeeGenome/GeneMark
export PERL5LIB=/home/xcao/p/anaconda3/lib/site_perl/5.26.2/
#install the required software and perl libs.
perl /home/xcao/p/gm_et_linux_64/gmes_petap/gmes_petap.pl --cores 48 --ES --sequence /lab02/Data_Raw/Xiaolong/20190616CooperationBeeGenome/Genome/hornetScaffolds.fasta
#after finished, transfer the file to caliburn

```

#### step 9 final run

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9
maker -CTL
```

modifly the maker_opts.ctl to
```
#-----Genome (these are always required)
genome=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/beeScaffolds.fasta #genome sequence (fasta file or fasta embeded in GFF3 file)
organism_type=eukaryotic #eukaryotic or prokaryotic. Default is eukaryotic

#-----Re-annotation Using MAKER Derived GFF3
maker_gff=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/maker_split_run/GENOME_GFF.all.gff #MAKER derived GFF3 file
est_pass=1 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=1 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no
protein_pass=1 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=1 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

#-----EST Evidence (for best results provide a file for at least one)
est= #set of ESTs or assembled mRNA-seq in fasta format
altest= #EST/cDNA sequence file in fasta format from an alternate organism
est_gff= #aligned ESTs or mRNA-seq from an external GFF3 file
altest_gff= #aligned ESTs from a closly relate species in GFF3 format

#-----Protein Homology Evidence (for best results provide a file for at least one)
protein=  #protein sequence file in fasta format (i.e. from mutiple oransisms)
protein_gff=  #aligned protein homology evidence from an external GFF3 file

#-----Repeat Masking (leave values blank to skip repeat masking)
model_org=all #select a model organism for RepBase masking in RepeatMasker
rmlib= #provide an organism specific repeat library in fasta format for RepeatMasker
repeat_protein= #provide a fasta file of transposable element proteins for RepeatRunner
rm_gff= #pre-identified repeat elements from an external GFF3 file
prok_rm=0 #forces MAKER to repeatmask prokaryotes (no reason to change this), 1 = yes, 0 = no
softmask=1 #use soft-masking rather than hard-masking in BLAST (i.e. seg and dust filtering)

#-----Gene Prediction
snaphmm=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/my_genome.hmm #SNAP HMM file
gmhmm=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step8/gmhmm.mod #GeneMark HMM file
augustus_species=bee20190616 #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no
trna=1 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no

#-----Other Annotation Feature Types (features MAKER doesn't recognize)
other_gff= #extra features to pass-through to final MAKER generated GFF3 file

#-----External Application Behavior Options
alt_peptide=C #amino acid used to replace non-standard amino acids in BLAST databases
cpus=1 #max number of cpus to use in BLAST and RepeatMasker (not for MPI, leave 1 when using MPI)

#-----MAKER Behavior Options
max_dna_len=100000 #length for dividing up contigs into chunks (increases/decreases memory usage)
min_contig=1 #skip genome contigs below this length (under 10kb are often useless)

pred_flank=200 #flank for extending evidence clusters sent to gene predictors
pred_stats=1 #report AED and QI statistics for all predictions as well as models
AED_threshold=1 #Maximum Annotation Edit Distance allowed (bound by 0 and 1)
min_protein=0 #require at least this many amino acids in predicted proteins
alt_splice=0 #Take extra steps to try and find alternative splicing, 1 = yes, 0 = no
always_complete=0 #extra steps to force start and stop codons, 1 = yes, 0 = no
map_forward=0 #map names and attributes forward from old GFF3 genes, 1 = yes, 0 = no
keep_preds=0 #Concordance threshold to add unsupported gene prediction (bound by 0 and 1)

split_hit=10000 #length for the splitting of hits (expected max intron size for evidence alignments)
single_exon=0 #consider single exon EST evidence when generating annotations, 1 = yes, 0 = no
single_length=250 #min length required for single exon ESTs if 'single_exon is enabled'
correct_est_fusion=0 #limits use of ESTs in annotation to avoid fusion genes

tries=2 #number of times to try a contig if there is a failure for some reason
clean_try=0 #remove all data from previous run before retrying, 1 = yes, 0 = no
clean_up=0 #removes theVoid directory with individual analysis files, 1 = yes, 0 = no
TMP= #specify a directory other than the system default temporary directory for temporary files
```

generate scripts
```python
total_seg = 1940
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/'
commands = f'{folder}cmds.txt'
fout = open(commands,'w')
for n in range(total_seg):
    genome_frag = f'/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/maker_split_run/{n}.all.gff'
    cmd = f'''source activate /gpfs/gpfs/scratch/xc278/maker2 && cd /home/scratch && mkdir beebee{n} && cd beebee{n} && cp {folder}maker*.ctl ./ && sed  -i 's/GENOME_GFF/{n}/g' maker_opts.ctl && maker -genome /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step1/maker_split_run/seq/{n}/{n} -cpus 1 &&  gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *.gff {folder}maker_split_run/ && fasta_merge -d *.maker.output/*master_datastore_index.log && mv *.fasta {folder}/maker_result_seqs && rm -rf /home/scratch/beebee{n} \n'''
    fout.write(cmd)
fout.close()
```

run scripts
```bash
srun -N 16 --partition long --qos long-award   -t 240:00:00 --pty bash #run twice to get 32 working nodes
python /home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py -t 24 -n e1c-051,e1c-052,e1c-053,e1c-054,e1c-055,e1c-056,e1c-057,e1c-058,e1c-059,e1c-060,e1c-061,e1c-062,e1c-063,e1c-064,e1c-065,e1c-066,e1c-109,e1c-110,e1c-111,e1c-112,e1c-113,e1c-114,e1c-115,e1c-116,e1c-117,e1c-118,e1c-119,e1c-120,e1c-121,e1c-122,e1c-123,e1c-124 -s 60 -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/cmds.txt 
```

collect data

```python
import glob
files = glob.glob('/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/maker_split_run/*.gff')
outfile = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/step9.all.gff'

ls_gff = []
ls_fasta = []
for f in files:
    txt = open(f).read()
    txt = txt.replace('##gff-version 3\n','',1)
    txt_gff, txt_fa = txt.split('##FASTA\n')
    ls_gff.append(txt_gff)
    ls_fasta.append(txt_fa)

fout = open(outfile,'w')
fout.write('##gff-version 3\n')
fout.write(''.join(ls_gff))
fout.write('##FASTA\n')
fout.write(''.join(ls_fasta))
fout.close()
```

collect protein and transcript sequence
```python
import glob
from Bio import SeqIO
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/'
files_pr = glob.glob(f'{folder}maker_result_seqs/*.maker.proteins.fasta')
files_dna = glob.glob(f'{folder}maker_result_seqs/*.maker.transcripts.fasta')
outfile_pr = f'{folder}step9.maker.proteins.fasta'
outfile_dna = f'{folder}step9.maker.transcripts.fasta'
def combineFa(files,outfile):
    '''
    combine fasta files and write outfile
    '''
    fout = open(outfile,'w')
    for f in files:
        for s in SeqIO.parse(f,'fasta'):
            fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
    fout.close()

combineFa(files_pr, outfile_pr)
combineFa(files_dna, outfile_dna)

```

### run maker2 for hornet

#### step1: first round of maker2

```bash
srun -N 1 --partition long --qos long-award   -t 240:00:00 --pty bash #allocate resource from Elf
source activate /gpfs/gpfs/scratch/xc278/maker2 #activate environment
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/ # go to the working directory
mkdir step1
cd step1
maker -CTL
```

modify the maker_opts.ctl file to 



```
#-----Genome (these are always required)
genome=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/hornetScaffolds.fasta #genome sequence (fasta file or fasta embeded in GFF3 file)
organism_type=eukaryotic #eukaryotic or prokaryotic. Default is eukaryotic

#-----Re-annotation Using MAKER Derived GFF3
maker_gff= #MAKER derived GFF3 file
est_pass=0 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=0 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no
protein_pass=0 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=0 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

#-----EST Evidence (for best results provide a file for at least one)
est=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/Trinity/Vespa_mandarinia_SRR2664950_Trinity.fa #set of ESTs or assembled mRNA-seq in fasta format
altest=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/ref.rna.faa #EST/cDNA sequence file in fasta format from an alternate organism
est_gff= #aligned ESTs or mRNA-seq from an external GFF3 file
altest_gff= #aligned ESTs from a closly relate species in GFF3 format

#-----Protein Homology Evidence (for best results provide a file for at least one)
protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/ref.proteins.faa  #protein sequence file in fasta format (i.e. from mutiple oransisms)
protein_gff=  #aligned protein homology evidence from an external GFF3 file

#-----Repeat Masking (leave values blank to skip repeat masking)
model_org=all #select a model organism for RepBase masking in RepeatMasker
rmlib= #provide an organism specific repeat library in fasta format for RepeatMasker
repeat_protein=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/ref/te_proteins.fasta #provide a fasta file of transposable element proteins for RepeatRunner
rm_gff= #pre-identified repeat elements from an external GFF3 file
prok_rm=0 #forces MAKER to repeatmask prokaryotes (no reason to change this), 1 = yes, 0 = no
softmask=1 #use soft-masking rather than hard-masking in BLAST (i.e. seg and dust filtering)

#-----Gene Prediction
snaphmm= #SNAP HMM file
gmhmm= #GeneMark HMM file
augustus_species= #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no
trna=1 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no

#-----Other Annotation Feature Types (features MAKER doesn't recognize)
other_gff= #extra features to pass-through to final MAKER generated GFF3 file

#-----External Application Behavior Options
alt_peptide=C #amino acid used to replace non-standard amino acids in BLAST databases
cpus=1 #max number of cpus to use in BLAST and RepeatMasker (not for MPI, leave 1 when using MPI)

#-----MAKER Behavior Options
max_dna_len=100000 #length for dividing up contigs into chunks (increases/decreases memory usage)
min_contig=1 #skip genome contigs below this length (under 10kb are often useless)

pred_flank=200 #flank for extending evidence clusters sent to gene predictors
pred_stats=0 #report AED and QI statistics for all predictions as well as models
AED_threshold=1 #Maximum Annotation Edit Distance allowed (bound by 0 and 1)
min_protein=0 #require at least this many amino acids in predicted proteins
alt_splice=0 #Take extra steps to try and find alternative splicing, 1 = yes, 0 = no
always_complete=0 #extra steps to force start and stop codons, 1 = yes, 0 = no
map_forward=0 #map names and attributes forward from old GFF3 genes, 1 = yes, 0 = no
keep_preds=0 #Concordance threshold to add unsupported gene prediction (bound by 0 and 1)

split_hit=10000 #length for the splitting of hits (expected max intron size for evidence alignments)
single_exon=0 #consider single exon EST evidence when generating annotations, 1 = yes, 0 = no
single_length=250 #min length required for single exon ESTs if 'single_exon is enabled'
correct_est_fusion=0 #limits use of ESTs in annotation to avoid fusion genes

tries=2 #number of times to try a contig if there is a failure for some reason
clean_try=0 #remove all data from previous run before retrying, 1 = yes, 0 = no
clean_up=0 #removes theVoid directory with individual analysis files, 1 = yes, 0 = no
TMP= #specify a directory other than the system default temporary directory for temporary files

```

split the genome to pieces ~100kb
```python
file_genome = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/hornetScaffolds.fasta'

import os
from Bio import SeqIO

n = 0
l = 0
outfolder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/genome_split/'
if not os.path.exists(outfolder):
    os.makedirs(outfolder)
outfile = open(os.path.join(outfolder,str(n)),'w')
for s in SeqIO.parse(file_genome,'fasta'):
    slen = len(s.seq)
    if slen <=200:
        continue
    l += slen
    outfile.write('>'+s.id+'\n'+str(s.seq)+'\n')
    if l >= 100000:
        outfile.close()
        n += 1
        l = 0
        outfile = open(os.path.join(outfolder,str(n)),'w')
outfile.close()
```

generate scripts
```python
total_seg = 2213
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/'
commands = f'{folder}cmds.txt'
fout = open(commands,'w')
for n in range(total_seg):
    cmd = f'source activate /gpfs/gpfs/scratch/xc278/maker2 && cd /home/scratch && mkdir {n} && cd {n} && cp {folder}/maker*.ctl ./ && maker -genome /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/genome_split/{n} -cpus 4 &&  gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *.gff {folder}/maker_split_run/ && rm -rf /home/scratch/{n} \n'
    fout.write(cmd)
fout.close()
```

Note: change cpus to 4, to speed up a little bit.

run the scripts manually in multiple nodes. Run on elf

```bash
ssh  -o "ServerAliveInterval 60" xc278@elf.rdi2.rutgers.edu
srun -N 16 --partition long --qos long-award   -t 240:00:00 --pty bash #run twice to get 32 working nodes
python /home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py -t 24 -n e1c-051,e1c-052,e1c-053,e1c-054,e1c-055,e1c-056,e1c-057,e1c-058,e1c-059,e1c-060,e1c-061,e1c-062,e1c-063,e1c-064,e1c-065,e1c-066,e1c-109,e1c-110,e1c-111,e1c-112,e1c-113,e1c-114,e1c-115,e1c-116,e1c-117,e1c-118,e1c-119,e1c-120,e1c-121,e1c-122,e1c-123,e1c-124 -s 60 -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/cmds.txt 
```

for some reason, the scripts stopped. find the unfinished fragements and re-run

```python
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/maker_split_run'
cmds = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/cmds_missing.txt'

import os
finished = os.listdir(folder)
#change output.all.gff to 0.all.gff
finished = [0] + [int(e.split('.')[0]) for e in finished]
missing = [e for e in range(2213) if e not in finished]
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/'
fout = open(cmds,'w')
for n in missing:
    cmd = f'source activate /gpfs/gpfs/scratch/xc278/maker2 && cd /home/scratch && mkdir {n} && cd {n} && cp {folder}/maker*.ctl ./ && maker -genome /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/genome_split/{n} -cpus 4 &&  gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *.gff {folder}/maker_split_run/ && rm -rf /home/scratch/{n} \n'
    fout.write(cmd)
fout.close()

```

run scripts

```bash
python /home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py -t 12 -n e1c-067,e1c-068,e1c-069,e1c-070,e1c-071,e1c-072,e1c-073,e1c-074,e1c-075,e1c-076,e1c-077,e1c-078,e1c-079,e1c-080,e1c-081,e1c-082 -s 60 -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/cmds_missing.txt &
```

Note, to be compatible with the following step, use python to combine the result

```python
import glob
files = glob.glob('/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/maker_split_run/*.gff')
outfile = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/step1.all.gff'

ls_gff = []
ls_fasta = []
for f in files:
    txt = open(f).read()
    txt = txt.replace('##gff-version 3\n','',1)
    txt_gff, txt_fa = txt.split('##FASTA\n')
    ls_gff.append(txt_gff)
    ls_fasta.append(txt_fa)

fout = open(outfile,'w')
fout.write('##gff-version 3\n')
fout.write(''.join(ls_gff))
fout.write('##FASTA\n')
fout.write(''.join(ls_fasta))
fout.close()
```

#### step2 train SNAP

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1
maker2zff step1.all.gff
fathom genome.ann genome.dna -validate > snap_validate_output.txt
cat snap_validate_output.txt | grep "error" #MODEL293 MODEL377 MODEL5879 with error, remove it
cat genome.ann | grep -vwE "MODEL293" |grep -vwE "MODEL377" |grep -vwE "MODEL5879" > genome.ann2
cat genome.ann2 |grep "MODEL293\|MODEL377\|MODEL5879"
cat genome.ann |grep "MODEL293\|MODEL377\|MODEL5879"
fathom genome.ann2 genome.dna -validate
fathom genome.ann2 genome.dna -categorize 1000
fathom uni.ann uni.dna -export 1000 -plus 
forge export.ann export.dna
hmm-assembler.pl my_genome . > my_genome.hmm
```

#### step3 Run MAKER with training results from Step2

Change the following line in the maker_opts.ctl file:

snaphmm=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/my_genome.hmm #SNAP HMM file

To base the predictions in the second MAKER run only on SNAP remove the filepaths to the protein and est evidence or set the flags for est2genome=0 and protein2genome=0.

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3
maker -CTL
```

change maker_opts.ctl to
```
#-----Genome (these are always required)
genome=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/hornetScaffolds.fasta #genome sequence (fasta file or fasta embeded in GFF3 file)
organism_type=eukaryotic #eukaryotic or prokaryotic. Default is eukaryotic

#-----Re-annotation Using MAKER Derived GFF3
maker_gff=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/maker_split_run/GENOME_GFF.all.gff #MAKER derived GFF3 file
est_pass=1 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=1 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no
protein_pass=1 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=1 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

#-----EST Evidence (for best results provide a file for at least one)
est= #set of ESTs or assembled mRNA-seq in fasta format
altest= #EST/cDNA sequence file in fasta format from an alternate organism
est_gff= #aligned ESTs or mRNA-seq from an external GFF3 file
altest_gff= #aligned ESTs from a closly relate species in GFF3 format

#-----Protein Homology Evidence (for best results provide a file for at least one)
protein=  #protein sequence file in fasta format (i.e. from mutiple oransisms)
protein_gff=  #aligned protein homology evidence from an external GFF3 file

#-----Repeat Masking (leave values blank to skip repeat masking)
model_org=all #select a model organism for RepBase masking in RepeatMasker
rmlib= #provide an organism specific repeat library in fasta format for RepeatMasker
repeat_protein= #provide a fasta file of transposable element proteins for RepeatRunner
rm_gff= #pre-identified repeat elements from an external GFF3 file
prok_rm=0 #forces MAKER to repeatmask prokaryotes (no reason to change this), 1 = yes, 0 = no
softmask=1 #use soft-masking rather than hard-masking in BLAST (i.e. seg and dust filtering)

#-----Gene Prediction
snaphmm=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/my_genome.hmm #SNAP HMM file #SNAP HMM file
gmhmm= #GeneMark HMM file
augustus_species= #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no
trna=1 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no

#-----Other Annotation Feature Types (features MAKER doesn't recognize)
other_gff= #extra features to pass-through to final MAKER generated GFF3 file

#-----External Application Behavior Options
alt_peptide=C #amino acid used to replace non-standard amino acids in BLAST databases
cpus=1 #max number of cpus to use in BLAST and RepeatMasker (not for MPI, leave 1 when using MPI)

#-----MAKER Behavior Options
max_dna_len=100000 #length for dividing up contigs into chunks (increases/decreases memory usage)
min_contig=1 #skip genome contigs below this length (under 10kb are often useless)

pred_flank=200 #flank for extending evidence clusters sent to gene predictors
pred_stats=0 #report AED and QI statistics for all predictions as well as models
AED_threshold=1 #Maximum Annotation Edit Distance allowed (bound by 0 and 1)
min_protein=0 #require at least this many amino acids in predicted proteins
alt_splice=0 #Take extra steps to try and find alternative splicing, 1 = yes, 0 = no
always_complete=0 #extra steps to force start and stop codons, 1 = yes, 0 = no
map_forward=0 #map names and attributes forward from old GFF3 genes, 1 = yes, 0 = no
keep_preds=0 #Concordance threshold to add unsupported gene prediction (bound by 0 and 1)

split_hit=10000 #length for the splitting of hits (expected max intron size for evidence alignments)
single_exon=0 #consider single exon EST evidence when generating annotations, 1 = yes, 0 = no
single_length=250 #min length required for single exon ESTs if 'single_exon is enabled'
correct_est_fusion=0 #limits use of ESTs in annotation to avoid fusion genes

tries=2 #number of times to try a contig if there is a failure for some reason
clean_try=0 #remove all data from previous run before retrying, 1 = yes, 0 = no
clean_up=0 #removes theVoid directory with individual analysis files, 1 = yes, 0 = no
TMP= #specify a directory other than the system default temporary directory for temporary files
```


generate scripts
```python
total_seg = 2213
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3/'
commands = f'{folder}cmds.txt'
fout = open(commands,'w')
for n in range(total_seg):
    genome_frag = f'/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step1/maker_split_run/{n}.all.gff'
    cmd = f'''source activate /gpfs/gpfs/scratch/xc278/maker2 && cd /home/scratch && mkdir hornet{n} && cd hornet{n} && cp {folder}maker*.ctl ./ && sed  -i 's/GENOME_GFF/{n}/g' maker_opts.ctl && maker -genome /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/genome_split/{n} -cpus 1 &&  gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *.gff {folder}maker_split_run/ && fasta_merge -d *.maker.output/*master_datastore_index.log && mv *.fasta {folder}/maker_result_seqs && rm -rf /home/scratch/hornet{n} \n'''
    fout.write(cmd)
fout.close()
```

run scripts
```bash
srun -N 16 --partition long --qos long-award   -t 240:00:00 --pty bash #run twice to get 32 working nodes
python /home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py -t 24 -n e1c-051,e1c-052,e1c-053,e1c-054,e1c-055,e1c-056,e1c-057,e1c-058,e1c-059,e1c-060,e1c-061,e1c-062,e1c-063,e1c-064,e1c-065,e1c-066,e1c-109,e1c-110,e1c-111,e1c-112,e1c-113,e1c-114,e1c-115,e1c-116,e1c-117,e1c-118,e1c-119,e1c-120,e1c-121,e1c-122,e1c-123,e1c-124 -s 60 -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3/cmds.txt 
```

collect data

```python
import glob
files = glob.glob('/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3/maker_split_run/*.gff')
outfile = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3/step3.all.gff'

ls_gff = []
ls_fasta = []
for f in files:
    txt = open(f).read()
    txt = txt.replace('##gff-version 3\n','',1)
    txt_gff, txt_fa = txt.split('##FASTA\n')
    ls_gff.append(txt_gff)
    ls_fasta.append(txt_fa)

fout = open(outfile,'w')
fout.write('##gff-version 3\n')
fout.write(''.join(ls_gff))
fout.write('##FASTA\n')
fout.write(''.join(ls_fasta))
fout.close()
```

#### step4 Train SNAP a second time with results from Step 3

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3
maker2zff step3.all.gff
fathom genome.ann genome.dna -validate > snap_validate_output.txt
cat snap_validate_output.txt | grep "error" #MODEL1963 MODEL6048 with error, remove them

grep -vwE "MODEL6048" genome.ann > genome.ann2
fathom genome.ann2 genome.dna -validate
fathom genome.ann2 genome.dna -categorize 1000
fathom uni.ann uni.dna -export 1000 -plus 
forge export.ann export.dna
hmm-assembler.pl my_genome . > my_genome.hmm
```

#### step6 train augustus
```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3
perl /gpfs/gpfs/scratch/xc278/maker2/bin/zff2augustus_gbk.pl > augustus.gbk
randomSplit.pl augustus.gbk 100
new_species.pl --species=hornet20190616
etraining --species=hornet20190616 augustus.gbk.train
augustus --species=hornet20190616 augustus.gbk.test | tee first_training.out
optimize_augustus.pl --species=hornet20190616 augustus.gbk.train --cpus=36 --kfold=36

```

#### step 8 train GeneMark

```bash
/home/xcao/p/gm_et_linux_64/gmes_petap/gmes_petap.pl --cores 48 --ES --sequence /lab02/Data_Raw/Xiaolong/20190616C                            ooperationBeeGenome/Genome/hornetScaffolds.fasta
```
Then move file to caliburn

#### step 9 final run

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/
cp /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3/maker*.ctl ./
mkdir maker_result_seqs
mkdir maker_split_run
```

modifly the maker_opts.ctl to
```
#-----Genome (these are always required)
genome=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/hornetScaffolds.fasta #genome sequence (fasta file or fasta embeded in GFF3 file)
organism_type=eukaryotic #eukaryotic or prokaryotic. Default is eukaryotic

#-----Re-annotation Using MAKER Derived GFF3
maker_gff=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3/maker_split_run/GENOME_GFF.all.gff #MAKER derived GFF3 file
est_pass=1 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=1 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no
protein_pass=1 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=1 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

#-----EST Evidence (for best results provide a file for at least one)
est= #set of ESTs or assembled mRNA-seq in fasta format
altest= #EST/cDNA sequence file in fasta format from an alternate organism
est_gff= #aligned ESTs or mRNA-seq from an external GFF3 file
altest_gff= #aligned ESTs from a closly relate species in GFF3 format

#-----Protein Homology Evidence (for best results provide a file for at least one)
protein=  #protein sequence file in fasta format (i.e. from mutiple oransisms)
protein_gff=  #aligned protein homology evidence from an external GFF3 file

#-----Repeat Masking (leave values blank to skip repeat masking)
model_org=all #select a model organism for RepBase masking in RepeatMasker
rmlib= #provide an organism specific repeat library in fasta format for RepeatMasker
repeat_protein= #provide a fasta file of transposable element proteins for RepeatRunner
rm_gff= #pre-identified repeat elements from an external GFF3 file
prok_rm=0 #forces MAKER to repeatmask prokaryotes (no reason to change this), 1 = yes, 0 = no
softmask=1 #use soft-masking rather than hard-masking in BLAST (i.e. seg and dust filtering)

#-----Gene Prediction
snaphmm=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3/my_genome.hmm #SNAP HMM file #SNAP HMM file
gmhmm=/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step8/gmhmm.mod #GeneMark HMM file
augustus_species=hornet20190616 #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no
trna=1 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no

#-----Other Annotation Feature Types (features MAKER doesn't recognize)
other_gff= #extra features to pass-through to final MAKER generated GFF3 file

#-----External Application Behavior Options
alt_peptide=C #amino acid used to replace non-standard amino acids in BLAST databases
cpus=1 #max number of cpus to use in BLAST and RepeatMasker (not for MPI, leave 1 when using MPI)

#-----MAKER Behavior Options
max_dna_len=100000 #length for dividing up contigs into chunks (increases/decreases memory usage)
min_contig=1 #skip genome contigs below this length (under 10kb are often useless)

pred_flank=200 #flank for extending evidence clusters sent to gene predictors
pred_stats=0 #report AED and QI statistics for all predictions as well as models
AED_threshold=1 #Maximum Annotation Edit Distance allowed (bound by 0 and 1)
min_protein=0 #require at least this many amino acids in predicted proteins
alt_splice=0 #Take extra steps to try and find alternative splicing, 1 = yes, 0 = no
always_complete=0 #extra steps to force start and stop codons, 1 = yes, 0 = no
map_forward=0 #map names and attributes forward from old GFF3 genes, 1 = yes, 0 = no
keep_preds=0 #Concordance threshold to add unsupported gene prediction (bound by 0 and 1)

split_hit=10000 #length for the splitting of hits (expected max intron size for evidence alignments)
single_exon=0 #consider single exon EST evidence when generating annotations, 1 = yes, 0 = no
single_length=250 #min length required for single exon ESTs if 'single_exon is enabled'
correct_est_fusion=0 #limits use of ESTs in annotation to avoid fusion genes

tries=2 #number of times to try a contig if there is a failure for some reason
clean_try=0 #remove all data from previous run before retrying, 1 = yes, 0 = no
clean_up=0 #removes theVoid directory with individual analysis files, 1 = yes, 0 = no
TMP= #specify a directory other than the system default temporary directory for temporary files
```

generate scripts
```python
total_seg = 2213
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/'
commands = f'{folder}cmds.txt'
fout = open(commands,'w')
for n in range(total_seg):
    cmd = f'''source activate /gpfs/gpfs/scratch/xc278/maker2 && cd /home/scratch && mkdir hornet{n} && cd hornet{n} && cp {folder}maker*.ctl ./ && sed  -i 's/GENOME_GFF/{n}/g' maker_opts.ctl && maker -genome /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/genome_split/{n} -cpus 4 &&  gff3_merge -d *.maker.output/*master_datastore_index.log &&  mv *.gff {folder}maker_split_run/ && fasta_merge -d *.maker.output/*master_datastore_index.log && mv *.fasta {folder}/maker_result_seqs && rm -rf /home/scratch/hornet{n} \n'''
    fout.write(cmd)
fout.close()
```

run scripts
```bash
srun -N 16 --partition long --qos long-award   -t 240:00:00 --pty bash #run twice to get 32 working nodes
python /home1/xc278/w/GitHub/xiaolongTools/multiThreadSlurm.py -t 24 -n e1c-051,e1c-052,e1c-053,e1c-054,e1c-055,e1c-056,e1c-057,e1c-058,e1c-059,e1c-060,e1c-061,e1c-062,e1c-063,e1c-064,e1c-065,e1c-066,e1c-109,e1c-110,e1c-111,e1c-112,e1c-113,e1c-114,e1c-115,e1c-116,e1c-117,e1c-118,e1c-119,e1c-120,e1c-121,e1c-122,e1c-123,e1c-124 -s 60 -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/cmds.txt 
```

collect data

```python
import glob
files = glob.glob('/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/maker_split_run/*.gff')
outfile = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/step9.all.gff'

ls_gff = []
ls_fasta = []
for f in files:
    txt = open(f).read()
    txt = txt.replace('##gff-version 3\n','',1)
    txt_gff, txt_fa = txt.split('##FASTA\n')
    ls_gff.append(txt_gff)
    ls_fasta.append(txt_fa)

fout = open(outfile,'w')
fout.write('##gff-version 3\n')
fout.write(''.join(ls_gff))
fout.write('##FASTA\n')
fout.write(''.join(ls_fasta))
fout.close()
```

collect protein and transcript sequence
```python
import glob
from Bio import SeqIO
folder = '/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/'
files_pr = glob.glob(f'{folder}maker_result_seqs/*.maker.proteins.fasta')
files_dna = glob.glob(f'{folder}maker_result_seqs/*.maker.transcripts.fasta')
outfile_pr = f'{folder}step9.maker.proteins.fasta'
outfile_dna = f'{folder}step9.maker.transcripts.fasta'
def combineFa(files,outfile):
    '''
    combine fasta files and write outfile
    '''
    fout = open(outfile,'w')
    for f in files:
        for s in SeqIO.parse(f,'fasta'):
            fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
    fout.close()

combineFa(files_pr, outfile_pr)
combineFa(files_dna, outfile_dna)

```

### BUSCO

install BUSCO

do the setting according to the manual

download the data set `wget http://busco.ezlab.org/v2/datasets/insecta_odb9.tar.gz`

#### run BUSCO for genome

```bash
cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/BUSCO/
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m genome -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/beeScaffolds.fasta -o 20190614beeGenome
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m genome -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/hornetScaffolds.fasta -o 20190614HornetGenome
```

result:

```
# BUSCO version is: 3.1.0 
# The lineage dataset is: insecta_odb9 (Creation date: 2016-02-13, number of species: 42, number of BUSCOs: 1658)
# To reproduce this run: python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/beeScaffolds.fasta -o 20190614beeGenome -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -m genome -c 36 -sp fly
#
# Summarized benchmarking in BUSCO notation for file /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/beeScaffolds.fasta
# BUSCO was run in mode: genome

	C:98.9%[S:98.8%,D:0.1%],F:1.0%,M:0.1%,n:1658

	1639	Complete BUSCOs (C)
	1638	Complete and single-copy BUSCOs (S)
	1	Complete and duplicated BUSCOs (D)
	16	Fragmented BUSCOs (F)
	3	Missing BUSCOs (M)
	1658	Total BUSCO groups searched




# BUSCO version is: 3.1.0 
# The lineage dataset is: insecta_odb9 (Creation date: 2016-02-13, number of species: 42, number of BUSCOs: 1658)
# To reproduce this run: python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/hornetScaffolds.fasta -o 20190614HornetGenome -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -m genome -c 36 -sp fly
#
# Summarized benchmarking in BUSCO notation for file /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/genome/hornetScaffolds.fasta
# BUSCO was run in mode: genome

	C:96.0%[S:95.0%,D:1.0%],F:3.0%,M:1.0%,n:1658

	1592	Complete BUSCOs (C)
	1575	Complete and single-copy BUSCOs (S)
	17	Complete and duplicated BUSCOs (D)
	49	Fragmented BUSCOs (F)
	17	Missing BUSCOs (M)
	1658	Total BUSCO groups searched
```

#### run BUSCO for maker result

cd /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/BUSCO/
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m prot -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/step9.maker.proteins.fasta -o 20190614beeMaker
python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -c 36 -m prot -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/step9.maker.proteins.fasta -o 20190614HornetMaker

result:

```
# BUSCO version is: 3.1.0 
# The lineage dataset is: insecta_odb9 (Creation date: 2016-02-13, number of species: 42, number of BUSCOs: 1658)
# To reproduce this run: python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/step9.maker.proteins.fasta -o 20190614beeMaker -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -m proteins -c 36
#
# Summarized benchmarking in BUSCO notation for file /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/step9.maker.proteins.fasta
# BUSCO was run in mode: proteins

	C:96.7%[S:96.6%,D:0.1%],F:1.1%,M:2.2%,n:1658

	1603	Complete BUSCOs (C)
	1602	Complete and single-copy BUSCOs (S)
	1	Complete and duplicated BUSCOs (D)
	19	Fragmented BUSCOs (F)
	36	Missing BUSCOs (M)
	1658	Total BUSCO groups searched


# BUSCO version is: 3.1.0 
# The lineage dataset is: insecta_odb9 (Creation date: 2016-02-13, number of species: 42, number of BUSCOs: 1658)
# To reproduce this run: python /home1/xc278/p/BUSCO/busco-master/scripts/run_BUSCO.py -i /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/step9.maker.proteins.fasta -o 20190614HornetMaker -l /home1/xc278/p/BUSCO/insecta/insecta_odb9/ -m proteins -c 36
#
# Summarized benchmarking in BUSCO notation for file /gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/step9.maker.proteins.fasta
# BUSCO was run in mode: proteins

	C:92.4%[S:90.7%,D:1.7%],F:2.9%,M:4.7%,n:1658

	1531	Complete BUSCOs (C)
	1503	Complete and single-copy BUSCOs (S)
	28	Complete and duplicated BUSCOs (D)
	48	Fragmented BUSCOs (F)
	79	Missing BUSCOs (M)
	1658	Total BUSCO groups searched


```

### save the results to ALU

on ALU server
```bash
cd /lab02/Data_Raw/Xiaolong/20190616CooperationBeeGenome
scp -r xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/BUSCO/ ./
scp  xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/Trinity/Vespa_mandarinia_SRR2664950_Trinity.fa ./TrinityHornet/
scp  xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step8/gmhmm.mod ./GeneMark/bee.gmhmm.mod
scp  xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step8/gmhmm.mod ./GeneMark/hornet.gmhmm.mod
scp  xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step3/my_genome.hmm ./snapHmm/bee.snap.gmhmm.mod
scp  xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step3/my_genome.hmm ./snapHmm/hornet.snap.gmhmm.mod
#augustus
scp -r xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/scratch/xc278/maker2/config/species/bee20190616/ ./AugustusHmm/
scp -r xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/scratch/xc278/maker2/config/species/hornet20190616/ ./AugustusHmm/

#maker
scp xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/step9.maker.proteins.fasta ./maker/20190617bee.maker.protein.fasta
scp xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/step9.maker.transcripts.fasta ./maker/20190617bee.maker.transcripts.fasta
scp xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/bee/step9/step9.all.gff ./maker/20190617bee.maker.gff

scp xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/step9.maker.proteins.fasta ./maker/20190617hornet.maker.protein.fasta
scp xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/step9.maker.transcripts.fasta ./maker/20190617hornet.maker.transcripts.fasta
scp xc278@caliburn.rdi2.rutgers.edu:/gpfs/gpfs/staging/jx76-003/xc/20190613cooperationBeeGenomes/maker2/hornet/step9/step9.all.gff ./maker/20190617hornet.maker.gff

cd maker/
pigz -k -9 *

```

